In [1]:
import memprocfs

In [2]:
vmm = memprocfs.Vmm(['-device', 'fpga', '-v', '-printf'])

VmmPyPluginLight: Register 'reg/net/bth_devices.txt'
VmmPyPluginLight: Register 'reg/net/tcpip_interfaces.txt'
VmmPyPluginLight: Register 'reg/usb/usb_devices.txt'
VmmPyPluginLight: Register 'reg/usb/usb_storage.txt'
VmmPyPluginLight: Register 'by-user/reg/user/wallpaper.txt'


In [3]:
process = vmm.process('Escape'+ 'FromTa'+'rkov.exe')
process.pid

20292

In [4]:
process.module

<function VmmProcess.module>

In [5]:
m = process.module("UnityPlayer.dll")
m

Module:20292:UnityPlayer.dll

In [6]:
hex(m.base)

'0x7ffbfb420000'

In [7]:
def read(addr: int, s: int) -> bytes: # read int 64
    return process.memory.read(addr, s)
def read_bool(addr: int) -> bool:
    return process.memory.read(addr, 1) == 0

def read_32(addr: int) -> int:
    addr = int.from_bytes(read(addr, 0x4), 'little')
    return addr

def read_64(addr: int) -> int:
    addr = int.from_bytes(read(addr, 0x8), 'little')
    return addr
def read_ptr(addr: int) -> int:
    addr = read_64(addr)
    if addr == 0x0:
        raise Exception("NullPointerError")
    return addr

def read_str(addr: int, s: int):
    res = read(addr, s)
    return res

def read_unity_str(addr):
    length = read_32(addr + Offsets.UnityString.Length)
    # PAGE_SIZE = 0x1000
    if length > 0x1000: raise DMAException("String length outside expected bounds!")
    buf = read(addr + Offsets.UnityString.Value, length*2)
    return buf


def read_ptr_chain(addr: int, offsets: list[int]):
    if len(offsets) == 0:
        return addr
    return read_ptr_chain(read_ptr(addr + offsets[0]), offsets[1:])

In [8]:
import struct
import json

class Offsets:
    class UnityList:
        Base = 0x10 # to UnityListBase
        Count = 0x18 # int32
    class UnityListBase:
        Start = 0x20 # start of list +(i * 0x8)
    class UnityString:
        Length = 0x10 # int32
        Value = 0x14 # string,unicode

    class ModuleBase:
        GameObjectManager = 0x17FFD28
    class GameObject:
        ObjectClass = 0x30
        ObjectName = 0x60 # string,default (null terminated)
    class GameWorld:
        # [Offsets.GameObject.ObjectClass, 0x18, 28]
        ToLocalGameWorld = [0x_30, 0x18, 0x28]
    class LocalGameWorld:
        ExfilController = 0x18
        LootList = 0x80
        RegisteredPlayers = 0xA0
        Grenades = 0x128

    class Player:
        # [ 0xA8, 0x28, 0x28, UnityList.Base, Offsets.UnityListBase.Start + (0 * 0x8), 0x10]
        To_TransformInternal = [ 0xA8, 0x28, 0x28, 0x_10, 0x_20 + (0 * 0x8), 0x10]
        MovementContext = 0x40
        Corpse = 0x350
        Profile = 0x520
        HealthController = 0x558
        InventoryController = 0x568
        IsLocalPlayer = 0x837

    class Profile:
        Id = 0x10
        AccountId = 0x18
        PlayerInfo = 0x28
        Stats = 0xE8

    class PlayerInfo:
        Nickname = 0x10 # unity string
        MainProfileNickname = 0x18 # unity string
        GroupId = 0x20 # ptr to UnityString (0/null if solo or bot)
        Settings = 0x48 # to PlayerSettings
        PlayerSide = 0x68 # int32
        RegDate = 0x6C # int32
        MemberCategory = 0x84 # int32 enum
        Experience = 0x88 # int32


    class HealthController:
        # EFT.HealthSystem.HealthValue -> Value (ValueStruct)
        To_HealthEntries = [0x58, 0x18] # to HealthEntries // if its wrong try { 0x50, 0x18 }

    class HealthEntry:
        Value = 0x10

    class HealthValue:
        Current = 0x0 # int32
        Maximum = 0x4 # int32
        Minimum = 0x8 # int32

    class TransformInternal:
        Hierarchy = 0x38 # to TransformHierarchy
        HierarchyIndex = 0x40 # 32

    class TransformHierarchy:
        Vertices = 0x18 # List<Vector128<float>>
        Indices = 0x20 # List<int>

In [9]:
gomp = int.from_bytes(process.memory.read(m.base + Offsets.ModuleBase.GameObjectManager, 0x8), 'little')
print("GameObjectManager addr: ", hex(gomp))
#q = m.memory.read(m.base, m.file_size)
#print(m.file_size)
#e = m.memory.read(m.base, 0x17FFD28 + 0xf0000)
#vmm.close()

GameObjectManager addr:  0x17a500099c0


In [10]:
class GameObjectManager:
    LastTaggedNode: int # 0x0
    TaggedNodes: int # 0x8
    LastMainCameraTaggedNode: int # 0x10
    MainCameraTaggedNodes: int # 0x18
    LastActiveNode: int # 0x20
    ActiveNodes: int # 0x28


    def __init__(self, addr: int):
        self.LastTaggedNode, self.TaggedNodes, \
        self.LastMainCameraTaggedNode, self.MainCameraTaggedNodes, \
        self.LastActiveNode, self.ActiveNodes \
        = struct.unpack('QQQQQQ', read(addr, 0x30))

gom = GameObjectManager(gomp)
vars(gom)

{'LastTaggedNode': 1642915312728,
 'TaggedNodes': 1634426405256,
 'LastMainCameraTaggedNode': 1639063549384,
 'MainCameraTaggedNodes': 1639063549384,
 'LastActiveNode': 1649562247032,
 'ActiveNodes': 1624842714488}

In [11]:
print(vmm.hex(read(gomp, 0x200)))

0000    58 34 62 85 7e 01 00 00  88 c1 67 8b 7c 01 00 00   X4b.~.....g.|...
0010    c8 f5 cc 9f 7d 01 00 00  c8 f5 cc 9f 7d 01 00 00   ....}.......}...
0020    78 5f 92 11 80 01 00 00  78 3d 2c 50 7a 01 00 00   x_......x=,Pz...
0030    90 03 97 fc fb 7f 00 00  e0 c7 b4 96 7d 01 00 00   ..... ......}...
0040    f8 01 00 00 00 00 00 00  63 00 52 00 29 00 00 00   ........c.R.)...
0050    29 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   )...............
0060    90 9f 00 50 7a 01 00 00  90 9f 00 50 7a 01 00 00   ...Pz......Pz...
0070    80 9a 00 50 7a 01 00 00  01 00 62 00 2e 00 57 00   ...Pz.....b...W.
0080    b0 31 59 fb fb 7f 00 00  72 00 20 00 38 00 00 00   .1Y.. ..r. .8...
0090    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00a0    30 7d 00 50 7a 01 00 00  02 00 00 00 00 00 00 00   0}.Pz...........
00b0    ff ff ff ff 00 00 00 00  72 00 20 00 39 00 00 00   ........r. .9...
00c0    90 9f 00 50 7a 01 00 00  20 9a 00 50 7a 01 00 00   ...Pz... ..Pz...
00d0    90 9

In [12]:
import re

class BaseObject:
    previousObjectLink: int # 0x0
    nextObjectLink: int # 0x8
    obj: int # 0x10 (to Offsets.GameObject)

    def __init__(self, addr: int):
        # print("Unpacking BaseObject: ", hex(addr), read(addr, 0x18))
        self.previousObjectLink, self.nextObjectLink, self.obj \
        = struct.unpack('QQQ', read(addr, 0x18))


def get_object_from_list(active_objects_ptr: int, last_object_ptr: int, object_name: str):
    active_object = BaseObject(read_ptr(read_ptr(active_objects_ptr)))
    last_object = BaseObject(read_ptr(read_ptr(last_object_ptr)))
    if active_object != 0:
        while active_object.obj != 0 and active_object.obj != last_object.obj:
            object_name_ptr = read_ptr(active_object.obj + Offsets.GameObject.ObjectName)
            object_name_str = read_str(object_name_ptr, 64)
            object_name_str = str(object_name_str.split(b"\x00")[0])
            if re.search(object_name, object_name_str, re.IGNORECASE):
                print("Found object: ", object_name_str)
                return active_object.obj
            active_object = BaseObject(active_object.nextObjectLink)
    print("Could not find object: ", object_name, "Maybe not in raid")
# gw = get_object_from_list(gom.ActiveNodes, gom.LastActiveNode, "GameWorld");

In [13]:
from time import sleep
from exceptions import *
import codecs


class Transform:
    _isPlayerTransform = False

    IndicesAddr: int # 64
    VerticesAddr: int # 64
    HierarchyIndex: int # 32

    def __init__(self, transform_internal: int, is_player_transform = False):
        hierarchy = read_64(transform_internal + Offsets.TransformInternal.Hierarchy)
        self.IndicesAddr = read_64(hierarchy + Offsets.TransformHierarchy.Indices)
        self.VerticesAddr = read_64(hierarchy + Offsets.TransformHierarchy.Vertices)
        self._isPlayerTransform = is_player_transform
        if is_player_transform:
            self.HierarchyIndex = 1
        else:
            self.HierarchyIndex = read_32(transform_internal + Offsets.TransformInternal.HierarchyIndex)

    # def get_position(self, obj: int):
    #     indices = []
    #     vertices = []
    #     if obj ==

class Player:
    # Player is a PMC Operator.
    Base: int
    Profile: int
    Position: (float, float) = (0, 0)
    Info: int
    HealthEntries: list[int]
    MovementContext: int
    TransformInternal: int
    _transform: Transform
    IsPmc: bool

    def get_account_id(self):
        id_ptr = read_ptr(self.Profile + Offsets.Profile.Id)
        return read_unity_str(id_ptr)

    def __init__(self, base: int, profile: int, pos = None):
        self.Base = base
        self.Profile = profile
        if pos is not None:
            self.Position = pos
        self.Info = read_ptr(profile + Offsets.Profile.PlayerInfo)
        health_entries_list = read_ptr_chain(base, [
            Offsets.Player.HealthController,
            Offsets.HealthController.To_HealthEntries[0],
            Offsets.HealthController.To_HealthEntries[1]
        ])
        self.HealthEntries = [0] * 7
        for i in range(7):
            self.HealthEntries[i] = read_ptr_chain(health_entries_list, [0x30 + (i*0x18), Offsets.HealthEntry.Value])

        self.MovementContext = read_ptr(base + Offsets.Player.MovementContext)
        self.TransformInternal = read_ptr(base + Offsets.Player.InventoryController)
        self._transform = Transform(self.TransformInternal, True)
        isLocalPlayer = read_bool(base + Offsets.Player.IsLocalPlayer)
        playerSide = read_32(self.Info + Offsets.PlayerInfo.PlayerSide) # Scav, PMC, etc.
        self.IsPmc = playerSide == 0x1 or playerSide == 0x2
        if isLocalPlayer:
            # Run this section while 'In-Raid' as a PMC (not Scav)
            print("LocalPlayer Acct Id: ", self.get_account_id())



class RegisteredPlayers:
    _base: int
    _listBase: int
    _players = dict()

    def get_player_count(self)->int:
        for i in range(5): # Re-attempt if read fails
            try:
                count = int.from_bytes(process.memory.read(self._base + Offsets.UnityList.Count, 0x4), 'little')
                if count < 1 or count > 1024:
                    raise Exception("ArgumentOutOfRangeException: ", count)
                return count
            except Exception as e:
                print("E: ", e)
                sleep(1000)
        return -1

    def update_list(self):
        count = self.get_player_count()
        if count < 1 or count > 1024:
            raise RaiseEnded()
        registered = set()
        for i in range(count):
            try:
                player_base = read_ptr(self._listBase + Offsets.UnityListBase.Start + i*0x8)
                # print(vmm.hex(read(player_base, 0x900)))
                player_profile = read_ptr(player_base + Offsets.Player.Profile)
                player_id = read_ptr(player_profile + Offsets.Profile.Id)
                player_id_len = read_32(player_id + Offsets.UnityString.Length)
                player_id_str = read_str(player_id + Offsets.UnityString.Value, player_id_len*2)

                if player_id_len != 24 and player_id_len != 27 and player_id_len != 36:
                    print(hex(player_base), hex(player_profile), hex(player_id), player_id_len, codecs.decode(player_id_str).replace('\x00', ''), "OUT OF RANGE")
                    raise Exception("ArgumentOutOfRangeException player_id_len")
                print(hex(player_base), hex(player_profile), hex(player_id), player_id_len, codecs.decode(player_id_str).replace('\x00', ''))
                registered.add(player_id_str)
                if self._players.get(player_id_str) is None:
                    player = Player(player_base, player_profile)
            except Exception as e:
                print("Failed to read player: ", e)
            # print(hex(player_base), hex(player_profile), hex(player_id), player_id_len, codecs.decode(player_id_str).replace('\x00', ''))

    # add stopwatch
    def __init__(self, base: int):
        self._base = base
        self._listBase = read_ptr(self._base + Offsets.UnityList.Base)

In [14]:
_rgtPlayers = None
_localGameWorld = None
def get_local_game_world():
    global _rgtPlayers, _localGameWorld
    game_world = get_object_from_list(gom.ActiveNodes, gom.LastActiveNode, "GameWorld")
    print("GameWorld: ", hex(game_world))
    if game_world == 0:
        print("Unable to find GameWorld Object, likely not in raid")
    _localGameWorld = read_ptr_chain(game_world, Offsets.GameWorld.ToLocalGameWorld)
    print("localGameWorld: ", hex(_localGameWorld))
    rgt_players = RegisteredPlayers(read_ptr(_localGameWorld + Offsets.LocalGameWorld.RegisteredPlayers))
    vars(rgt_players)
    if rgt_players.get_player_count() > 1:
        _rgtPlayers = rgt_players
        return True

In [15]:
get_local_game_world()

Found object:  b'GameWorld'
GameWorld:  0x17a9b26cc60
localGameWorld:  0x17d7bf6c400


True

In [16]:
_rgtPlayers.get_player_count()

21

In [17]:
_rgtPlayers.update_list()

Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError
Failed to read player:  NullPointerError


In [69]:
size = 10*0x8
# gomb = process.memory.read(gom, 0x30)

# hex(ActiveNodes)
# scatter = process.memory.scatter_initialize(memprocfs.FLAG_NOCACHE)
# scatter

In [253]:
# scatter.prepare(gom, size)
# scatter.execute()
_

'0x1790e039e30'

In [80]:
# an = (scatter.read(gom+0x28, 0x8)) #ActiveNodes
# print(vmm.hex(scatter.read(gom, 0x30)))
# print(scatter.read(gom+0x9, 0x8))
# size
# scatter.close()

0000    c8 20 f9 e1 76 01 00 00  e8 10 ed 92 73 01 00 00   . ..v.......s...
0010    c8 77 a0 09 72 01 00 00  c8 77 a0 09 72 01 00 00   .w..r....w..r...
0020    48 63 23 54 78 01 00 00  38 9f 3f 00 72 01 00 00   Hc#Tx...8.?.r...

b''


In [18]:
vmm.close()

In [64]:
sb

b''

In [65]:
q

NameError: name 'q' is not defined

In [54]:
m.base

140718532919296

In [31]:
0xf0000

983040

In [52]:
with open("bin3", "wb") as bk:
    bk.write(b)
    bk.close()

OSError: [Errno 22] Invalid argument: 'bin3'

In [42]:
process

Process:5364

In [50]:
b.find("Herr")

-1